In [3]:
import keras
from keras.models import load_model
import random
import numpy as np
import math
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import adam_v2
from IPython.display import clear_output
import time
from pandas_datareader import data
from datetime import datetime
import matplotlib.pyplot as plt
import pandas as pd
from keras import backend as K
import tensorflow as tf
import ta

import statistics

In [4]:
# prints formatted price
def formatPrice(n):
	return ("-$" if n < 0 else "$") + "{0:.2f}".format(abs(n))

In [5]:
X1=pd.read_csv('csv/DJI.csv')

In [6]:
x=X1["Close"][3550] #testing from closing price 3550 onwards 

In [7]:
X1['rsi']=ta.momentum.RSIIndicator(X1["Close"], window=14, fillna=True).rsi()
X1['EMA']=ta.trend.EMAIndicator(X1["Close"], window=10, fillna=True).ema_indicator()
X1['ATR']=ta.volatility.average_true_range(high=X1["High"], low=X1["Low"], close=X1["Close"], window=14, fillna=True)

In [8]:
def tech(t):
    ls=[0,0,0]
    #0-hold 1-buy 2-sell
    rsi=X1['rsi'][:t+1] #trading-decision based on RSI
    if(rsi[t]>=70):
        ls[0]=2
    elif (rsi[t]<=30):
         ls[0]=1
    else:
         ls[0]=0
    ema=X1['EMA'][:t+1] #trading-decision based on EMA
    if(X1['Low'][t] > ema[t]): #i.e. the ema is lower than the low of the current candle
        ls[1]=2
    elif (X1['High'][t] < ema[t]): #i.e. the ema is higher than the high of the current candle
         ls[1]=1
    else:
         ls[1]=0
    atr=X1['ATR'][:t+1] #trading-decision based on ATR
    if(atr[t] > statistics.mean(atr[t-20:t])*1.3): #i.e. the current value of the ATR is at least 30% higher than the average of the last 20 ATR-Values
        ls[2]=2
    elif (atr[t] > statistics.mean(atr[t-20:t])*1.3): #i.e. the current value of the ATR is at least 30% higher than the average of the last 20 ATR-Values
         ls[2]=1
    else:
         ls[2]=0
    return ls

In [9]:
class Portfolio:
    def __init__(self):
        self.cash=10000000
        self.profit=0
        self.inventory = []
        self.nshare=0
        self.last_price=0
        self.sold_share=0
        
    def getData(self):        
        self.vec = []
        self.df=pd.read_csv('csv/DJI.csv')
        return self.df 
    
    def getState(self,data,t):
        self.n=50
        self.res=[]
        for i in range(self.n):                 
            self.res.append(data['Close'][t-i])          
        return np.array([self.res])
        

In [10]:
class RLAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95    # discount rate
        self.learning_rate = 0.001
        self.model = load_model('models/trainedmodel') #loads trained model for testing on teesting data
        self.target_model = self._model()
        self.update_target_model()
      

    def _model(self):
        model = Sequential()
        model.add(Dense(units=64, input_dim=self.state_size, activation="relu"))
        model.add(Dense(units=32, activation="relu"))
        model.add(Dense(units=8, activation="relu"))
        model.add(Dense(self.action_size, activation="linear"))
        model.compile(loss="mse",optimizer=adam_v2.Adam(learning_rate=0.001))
        return model

    def update_target_model(self):
        # copy weights from model to target_model
        self.target_model.set_weights(self.model.get_weights())

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])  # returns action

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = self.model.predict(state)
            if done:
                target[0][action] = reward
            else:
                #a = self.model.predict(next_state)[0]
                t = self.target_model.predict(next_state)[0]
                target[0][action] = reward + self.gamma * np.amax(t)
                #target[0][action] = reward + self.gamma * t[np.argmax(a)]
            self.model.fit(state, target, epochs=1, verbose=1)

In [11]:
rl = RLAgent(50,3)
episode_count=1
for e in range(episode_count):
    agent=Portfolio()
    data=agent.getData()
    data['Action']=""
    data['actual_action']=""
    data['share']=""
    data['Cash']=""
    data['trade_profit']=""
    data['Adj_cash']=""
    l = len(data) - 1
    batch_size = 32
    print( "Episode " + str(e+1) + "/" + str(episode_count))
    state = agent.getState(data, 3550)
    initial_cash=agent.cash
    total_profit = 0
    total_brokerage=0
    agent.nshare=0
    agent.last_price=0
    data['Action'][3550]=0
    data['share'][3550]=agent.nshare
    data['Cash'][3550]= agent.cash
    pos=""
   
    for t in range(3550,l):
        z=tech(t)
        z0=z.count(0)
        z1=z.count(1)
        z2=z.count(2)
        y=[z0,z1,z2]
        z3=y.index(max(y))
        action = rl.act(state)
        data['Action'][t]=0
        data['share'][t]=agent.nshare
        data['Cash'][t]= agent.cash
            
        # sit
        next_state = agent.getState(data, t + 1)
        reward = 0
        if action == 0:
            data['Action'][t]=0
            if agent.nshare == 0:
                    data['Adj_cash'][t]=agent.cash
            elif agent.nshare >0:
                    data['Adj_cash'][t]=agent.cash+agent.nshare*data['Close'][t]
            elif agent.nshare <0:
                    data['Adj_cash'][t]=(agent.cash+(-agent.sold_share)*agent.last_price)+(agent.last_price - data['Close'][t])*(-agent.sold_share)
                    
        elif action == 1:# buy
                data['Action'][t]=1
                
                if z3==1:                            
                    if  agent.nshare == 0: #long buy
                        agent.nshare=int(agent.cash/data['Close'][t])
                        agent.cash=agent.cash-agent.nshare*data['Close'][t]
                        agent.last_price=data['Close'][t]
                        data['actual_action'][t]="Bought-open-long"
                        data['share'][t]=agent.nshare
                        data['Cash'][t]= agent.cash
                    elif agent.nshare < 0: #short buy
                        
                        sell_price=agent.last_price
                        ns = agent.nshare
                        agent.nshare=0
                        agent.last_price=0
                        brokerage=((sell_price + data['Close'][t])*(-ns))*0.0010
                        total_brokerage=total_brokerage+brokerage
                        data['trade_profit'][t]=((sell_price - data['Close'][t])*(-ns))-brokerage
                        total_profit = total_profit + ((sell_price - data['Close'][t])*(-ns))-brokerage
                        agent.cash=agent.cash+(-ns*sell_price)+data['trade_profit'][t]
                        data['actual_action'][t]="Bought-closed-short"
                        agent.sold_share=0
                
                    
                    data['Adj_cash'][t]=agent.cash+agent.nshare*data['Close'][t]
                    reward = (1+((data['Close'][t]-data['Close'][t-1])/data['Close'][t-1])*(data['Close'][t-1]/data['Close'][t-50]))
                    data['share'][t]=agent.nshare
                    data['Cash'][t]= agent.cash
                else:
                    if agent.nshare == 0:
                        data['Adj_cash'][t]=agent.cash
                    elif agent.nshare >0:
                        data['Adj_cash'][t]=agent.cash+agent.nshare*data['Close'][t]
                    elif agent.nshare <0:
                        data['Adj_cash'][t]=(agent.cash+(-agent.sold_share)*agent.last_price)+(agent.last_price - data['Close'][t])*(-agent.sold_share)
                                                      
        elif action == 2:# sell
            data['Action'][t]=2
            
            if z3==2:  
                    if  agent.nshare == 0: #short sell
                        agent.nshare=int(agent.cash/data['Close'][t])
                             
                        agent.cash=agent.cash-agent.nshare*data['Close'][t]
                        agent.nshare= -agent.nshare
                        agent.sold_share=agent.nshare
                        agent.last_price=data['Close'][t]
                        data['actual_action'][t]="Sold-open-short"
                                
                    elif agent.nshare > 0: 
                        
                        buy_price=agent.last_price
                        ns = agent.nshare
                        agent.nshare=0
                        agent.last_price=0
                        brokerage=((buy_price + data['Close'][t])*(ns))*0.0010
                        data['trade_profit'][t]=(data['Close'][t]-buy_price)*(ns)-brokerage
                        total_profit = total_profit + (data['Close'][t]-buy_price)*(ns)-brokerage
                        agent.cash=agent.cash+(ns)*data['Close'][t]-brokerage
                        data['actual_action'][t]="Sold-closed-long"
               
                    
                    data['Adj_cash'][t]=agent.cash+(-agent.nshare)*agent.last_price+(agent.last_price - data['Close'][t])*(-agent.nshare)
                    reward = -1*(1+((data['Close'][t]-data['Close'][t-1])/data['Close'][t-1])*(data['Close'][t-1]/data['Close'][t-50]))
                    data['share'][t]=agent.nshare
                    data['Cash'][t]= agent.cash
            else:
                if agent.nshare == 0:
                    data['Adj_cash'][t]=agent.cash
                elif agent.nshare >0:
                    data['Adj_cash'][t]=agent.cash+agent.nshare*data['Close'][t]
                elif agent.nshare <0:
                    data['Adj_cash'][t]=(agent.cash+(-agent.nshare)*agent.last_price)+(agent.last_price - data['Close'][t])*(-agent.nshare)
                    
        done = True if t == l - 1 else False
        rl.remember(state, action, reward, next_state, done)
        state = next_state

        if done:
                rl.update_target_model()
                print( "------------------------------------------")
                print ("Initial Cash: " + formatPrice(initial_cash))
                print ("Final Cash: " + formatPrice(data['Adj_cash'][t]))
                print ("Total  Profit: " + formatPrice(data['Adj_cash'][t]-initial_cash))
                print ("Total % Profit:{} ".format((data['Adj_cash'][t]-initial_cash)/initial_cash*100))
                print("-------------------------------------------")

        if len(rl.memory) > batch_size:
            rl.replay(batch_size)
            
    if e % 1 == 0:
        rl.model.save("models/trainedmodel0" + str(e))
        data.to_csv('csv/finalmodel_data.csv'+ str(e))

Episode 1/1


<ipython-input-11-c01c86aa2717>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Action'][3550]=0
<ipython-input-11-c01c86aa2717>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['share'][3550]=agent.nshare
<ipython-input-11-c01c86aa2717>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Cash'][3550]= agent.cash
<ipython-input-11-c01c86aa2717>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sl

1/1 [==============================] - 0s 4ms/step - loss: 20.0991


1/1 [==============================] - 0s 0s/step - loss: 0.0044


1/1 [==============================] - 0s 0s/step - loss: 0.3014


...

1/1 [==============================] - 0s 0s/step - loss: 2.7665e-04


1/1 [==============================] - 0s 0s/step - loss: 0.0075


1/1 [==============================] - 0s 0s/step - loss: 0.1039


1/1 [==============================] - 0s 0s/step - loss: 0.0816


1/1 [==============================] - 0s 0s/step - loss: 0.1074
------------------------------------------
Initial Cash: $10000000.00
Final Cash: $7126422.06
Total  Profit: -$2873577.94
Total % Profit:-28.735779374999996 
-------------------------------------------
1/1 [==============================] - 0s 4ms/step - loss: 53.5956
INFO:tensorflow:Assets written to: models/trainedmodel00\assets
